In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

date_cols = ['step_start_date','latest_rental_date','rental_created_date']

df = pd.read_csv('ivr_prioritization_7_8_21.csv')
df_retention = pd.read_csv('ivr_prioritization_7_8_21.csv')
for col in date_cols:
    df[col] = pd.to_datetime(df[col],errors='ignore')
    df_retention[col] = pd.to_datetime(df_retention[col],errors='ignore')

print(df.info())
print(df_retention.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65573 entries, 0 to 65572
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   exit                 65573 non-null  object        
 1   id                   65573 non-null  object        
 2   user_id              64916 non-null  float64       
 3   tag                  65111 non-null  object        
 4   tag_category         65118 non-null  object        
 5   csat                 10338 non-null  float64       
 6   csat_returned        65573 non-null  int64         
 7   step_start_date      65573 non-null  datetime64[ns]
 8   latest_rental_date   63335 non-null  datetime64[ns]
 9   rental_created_date  64858 non-null  datetime64[ns]
 10  hold_time            65573 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int64(1), object(4)
memory usage: 5.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65573 entries,

In [69]:
df = df.drop_duplicates(subset='id',keep='first')
print(df.duplicated(subset='id').sum())
print(df.info())

0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21186 entries, 0 to 65572
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   exit                 21186 non-null  object        
 1   id                   21186 non-null  object        
 2   user_id              20537 non-null  float64       
 3   tag                  20999 non-null  object        
 4   tag_category         21003 non-null  object        
 5   csat                 3623 non-null   float64       
 6   csat_returned        21186 non-null  int64         
 7   step_start_date      21186 non-null  datetime64[ns]
 8   latest_rental_date   19076 non-null  datetime64[ns]
 9   rental_created_date  20479 non-null  datetime64[ns]
 10  hold_time            21186 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int64(1), object(4)
memory usage: 1.9+ MB
None


In [70]:
print(df.groupby('exit')['csat_returned'].value_counts())

exit               csat_returned
no-match           0                13880
                   1                 2809
power_user_no_vip  0                 1915
                   1                  337
power_user_vip     0                 1768
                   1                  477
Name: csat_returned, dtype: int64


In [71]:
df['user_id'] = df['user_id'].fillna(0)
df['user_id'].isnull().sum()

0

In [72]:
df.isnull().sum()

exit                       0
id                         0
user_id                    0
tag                      187
tag_category             183
csat                   17563
csat_returned              0
step_start_date            0
latest_rental_date      2110
rental_created_date      707
hold_time                  0
dtype: int64

In [73]:
csat_returned = df[df['csat_returned']==1]
print(csat_returned.head())

               exit                  id  user_id               tag  \
177        no-match  5003o00001TfhouAAB      0.0            Credit   
323        no-match  5003o00001SgUDdAAN      0.0        Promo Code   
435  power_user_vip  5003o00001TfcPaAAJ      0.0  Operator Contact   
489        no-match  5003o00001TgKFtAAN      0.0           Booking   
551        no-match  5003o00001SiAkcAAF      0.0   General Billing   

           tag_category  csat  csat_returned step_start_date  \
177  Customer Questions   5.0              1      2021-06-30   
323  Customer Questions   5.0              1      2021-05-29   
435              Supply   1.0              1      2021-06-29   
489  Customer Questions   5.0              1      2021-07-08   
551             Billing   1.0              1      2021-06-21   

    latest_rental_date rental_created_date   hold_time  
177                NaT                 NaT  437.286002  
323                NaT                 NaT    0.000000  
435                NaT 

In [8]:
csat_returned.shape

(3623, 11)

In [9]:
import scipy.stats as stats

stats.f_oneway(csat_returned['csat'][csat_returned['exit'] == 'no-match'],
               csat_returned['csat'][csat_returned['exit'] == 'power_user_vip'],
               csat_returned['csat'][csat_returned['exit'] == 'power_user_no_vip'])

F_onewayResult(statistic=18.738231594025844, pvalue=8.015400914003312e-09)

In [10]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model1 = ols('csat ~ C(exit)', data=csat_returned).fit()
aov_table = sm.stats.anova_lm(model1, typ=2)
aov_table

C:\Users\Jimmy Reagan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


,sum_sq,df,F,PR(>F)
C(exit),60.84549,2.0,18.738232,8.015401e-09
Residual,5877.30687,3620.0,NaN,NaN


In [11]:
stats.shapiro(model1.resid)

(0.5439094305038452, 0.0)

In [12]:
stats.levene(csat_returned['csat'][csat_returned['exit'] == 'no-match'],
               csat_returned['csat'][csat_returned['exit'] == 'power_user_vip'],
               csat_returned['csat'][csat_returned['exit'] == 'power_user_no_vip'])

LeveneResult(statistic=18.738231594025848, pvalue=8.015400914003312e-09)

group1,group2,meandiff,lower,upper,reject
no-match,power_user_no_vip,-0.1321,-0.3044,0.0401,False
no-match,power_user_vip,0.3522,0.2043,0.5002,True
power_user_no_vip,power_user_vip,0.4843,0.2717,0.6969,True


In [14]:
comp2 = mc.MultiComparison(csat_returned['csat'], csat_returned['exit'])
tbl1, a1, a2 = comp2.allpairtest(stats.ttest_ind, method= "bonf")

tbl1

group1,group2,stat,pval,pval_corr,reject
no-match,power_user_no_vip,1.7295,0.0838,0.2514,False
no-match,power_user_vip,-5.6651,0.0,0.0,True
power_user_no_vip,power_user_vip,-5.9646,0.0,0.0,True


In [15]:
comp3 = mc.MultiComparison(csat_returned['csat'], csat_returned['exit'])
tbl2, a3, a4 = comp3.allpairtest(stats.ttest_ind, method= "sidak")

tbl2

group1,group2,stat,pval,pval_corr,reject
no-match,power_user_no_vip,1.7295,0.0838,0.2309,False
no-match,power_user_vip,-5.6651,0.0,0.0,True
power_user_no_vip,power_user_vip,-5.9646,0.0,0.0,True


In [16]:
w, pvalue = stats.shapiro(model1.resid)
print(w, pvalue)

0.5439094305038452 0.0


In [17]:
from bioinfokit.analys import stat 
res = stat()
res.levene(df=csat_returned, res_var='csat', xfac_var='exit')
res.levene_summary

ModuleNotFoundError: No module named 'bioinfokit'

In [18]:
stats.f_oneway(df['csat_returned'][df['exit'] == 'no-match'],
               df['csat_returned'][df['exit'] == 'power_user_vip'],
               df['csat_returned'][df['exit'] == 'power_user_no_vip'])

F_onewayResult(statistic=17.69230215284222, pvalue=2.1025249599805373e-08)

In [19]:
comp4 = mc.MultiComparison(df['csat_returned'], df['exit'])
post_hoc_res2 = comp4.tukeyhsd()
post_hoc_res2.summary()

group1,group2,meandiff,lower,upper,reject
no-match,power_user_no_vip,-0.0187,-0.0385,0.0011,False
no-match,power_user_vip,0.0442,0.0243,0.064,True
power_user_no_vip,power_user_vip,0.0628,0.0365,0.0891,True


In [20]:
comp5 = mc.MultiComparison(df['csat_returned'], df['exit'])
tbl3, a5, a6 = comp5.allpairtest(stats.ttest_ind, method= "bonf")

tbl3

group1,group2,stat,pval,pval_corr,reject
no-match,power_user_no_vip,2.2348,0.0254,0.0763,False
no-match,power_user_vip,-5.1901,0.0,0.0,True
power_user_no_vip,power_user_vip,-5.4884,0.0,0.0,True


In [21]:
categories = ['Facility', 'Customer Questions', 'User Error', 'Reservation Actions', 'Other', 'SH Error']
tag = ['Lot Full', 'Booking', 'Equipment Error', 'Change of Plans', 'Redemption', 'Booked By Mistake', 'Change Date / Time']

def filtered_anova(col_name,list_name):
    for name in list_name:
        print(name + ' ANOVA')
        temp_df = csat_returned[csat_returned[col_name] == name]
        temp_comp = mc.MultiComparison(temp_df['csat'], temp_df['exit'])
        temp_hoc_res = temp_comp.tukeyhsd(alpha = 0.05)
        print(temp_hoc_res.summary())
        
filtered_anova('tag_category', categories)
filtered_anova('tag', tag)


Facility ANOVA
        Multiple Comparison of Means - Tukey HSD,FWER=0.05        
      group1            group2      meandiff  lower  upper  reject
------------------------------------------------------------------
     no-match     power_user_no_vip  0.217   -0.1129 0.5469 False 
     no-match       power_user_vip   0.7388   0.4776 0.9999  True 
power_user_no_vip   power_user_vip   0.5217   0.1392 0.9043  True 
------------------------------------------------------------------
Customer Questions ANOVA
        Multiple Comparison of Means - Tukey HSD,FWER=0.05        
      group1            group2      meandiff  lower  upper  reject
------------------------------------------------------------------
     no-match     power_user_no_vip -0.3682  -0.7903 0.0538 False 
     no-match       power_user_vip  -0.2496  -0.6802 0.181  False 
power_user_no_vip   power_user_vip   0.1187  -0.4609 0.6983 False 
------------------------------------------------------------------
User Error ANOVA
     

In [22]:
comp6 = mc.MultiComparison(df['hold_time'], df['exit'])
post_hoc_res3 = comp6.tukeyhsd()
post_hoc_res3.summary()

group1,group2,meandiff,lower,upper,reject
no-match,power_user_no_vip,-3.8811,-18.8549,11.0926,False
no-match,power_user_vip,-14.3246,-29.3189,0.6698,False
power_user_no_vip,power_user_vip,-10.4434,-30.3363,9.4495,False


In [74]:
df_retention['rental_created_date'] = df_retention['rental_created_date'].fillna('1/1/2021')
df_retention['rental_created_date'] = pd.to_datetime(df_retention['rental_created_date'])
df_retention['retention_rentals'] = df_retention['rental_created_date'].apply(lambda x: 1 if df_retention['rental_created_date'] > df_retention['step_start_date'] else 0)
print(df_retention[df_retention['retention_rentals'] == 1].head(20))

#df_retention['rental_created_date'] = df_retention['rental_created_date'].fillna(0)
#print(df_retention['rental_created_date'].tail(50))
#print(df_retention.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65573 entries, 0 to 65572
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   exit                 65573 non-null  object        
 1   id                   65573 non-null  object        
 2   user_id              64916 non-null  float64       
 3   tag                  65111 non-null  object        
 4   tag_category         65118 non-null  object        
 5   csat                 10338 non-null  float64       
 6   csat_returned        65573 non-null  int64         
 7   step_start_date      65573 non-null  datetime64[ns]
 8   latest_rental_date   63335 non-null  datetime64[ns]
 9   rental_created_date  65573 non-null  datetime64[ns]
 10  hold_time            65573 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int64(1), object(4)
memory usage: 5.5+ MB
None


In [91]:

df_retention['retention_rentals'] = df_retention['rental_created_date'] > df_retention['step_start_date'] 

print(df_retention[df_retention['retention_rentals'] == True].head(30))

                  exit                  id   user_id                tag  \
658  power_user_no_vip  5003o00001SiVJvAAN  999983.0            Booking   
667           no-match  5003o00001SiSf9AAF  999876.0  Booked By Mistake   
672  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
673  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
674  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
675  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
676  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
677  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
678  power_user_no_vip  5003o00001SfwzPAAR  999862.0  Booked By Mistake   
681           no-match  5003o00001SguVVAAZ  999708.0    Equipment Error   
684  power_user_no_vip  5003o00001Sg6MIAAZ  998390.0            Booking   
685           no-match  5003o00001Sg6LVAAZ  998390.0          Duplicate   
686  power_user_no_vip  5

In [96]:
df_grouped = df_retention.groupby('user_id').agg({'user_id':'count','retention_rentals':'sum'})
print(df_grouped)

           user_id  retention_rentals
user_id                              
3979.0           1                  0
3982.0           4                  1
4322.0           1                  1
5275.0           2                  0
5318.0           4                  3
...            ...                ...
8542225.0        1                  0
8542440.0        1                  0
8542489.0        1                  0
8542491.0        1                  0
8542576.0        1                  0

[17390 rows x 2 columns]


In [97]:
df_grouped.describe()

,user_id,retention_rentals
count,17390.000000,17390.000000
mean,3.732950,1.227487
std,7.089026,3.817341
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,4.000000,1.000000
max,240.000000,103.000000
